In [ ]:
# =============================================================================
# Basic functions for running FOOOF across a group of power spectra.
#
# Created by Douglas Angus, Bond University, 2023.
# =============================================================================

In [2]:
import numpy as np
import scipy.io as spio
import sys
import matplotlib.pyplot as plt
import fooof
from fooof import FOOOFGroup
import pandas as pd
import os as os

def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
def fooof_loop(subjects, condition, home_dir, file_suffix):
    """
	Loop through subjects and fit FOOOF model to each subject's power spectrum
	"""

    #Establish dictionary
    results = {}
    results['ID'] = []
    results['exponent'] = []
    results['offset'] = []
    results['r2'] = []
        
    for s in range(0, np.size(subjects)):
        sub = subjects[s]
        
        fname = os.path.join(home_dir, (str(sub)+ file_suffix + '.mat'))
        dat = spio.loadmat(fname, mdict=None, mat_dtype=True, struct_as_record=False)

        ps = dat['SpectraSheetSubject']
        ps = 10**(ps/10)
                
        # first initialize fooof group object
        fg = FOOOFGroup(peak_width_limits=[1, 8], min_peak_height=0.1, max_n_peaks=8, aperiodic_mode='fixed')
        freq_range = [2, 40]

        freqs = np.linspace(1,100, num= np.size(ps,1))

        fg.fit(freqs, ps, freq_range) 
        exps_temp = fg.get_params('aperiodic_params', 'exponent')
        offset_temp = fg.get_params('aperiodic_params', 'offset')
        r_squ_temp = fg.get_params('r_squared')

        # Store results in dictionary
        results['ID'].append(sub)
        results['exponent'].append(exps_temp[np.newaxis])
        results['offset'].append(offset_temp[np.newaxis])
        results['r2'].append(r_squ_temp[np.newaxis])
        
    return results

In [ ]:
good_subs = np.array(['sub-010002',	'sub-010004'])

home_dir = os.path.realpath(os.path.join(os.path.abspath(''), 'Data'))

#Run fooof loops over EC and EO data
import warnings
warnings.filterwarnings('ignore')

file_suffix = '_EC'
EC_results = fooof_loop(good_subs, "eyes_closed", home_dir, file_suffix)

file_suffix = '_EO'
EO_results = fooof_loop(good_subs, "eyes_open", home_dir, file_suffix)

np.save(os.path.join(home_dir,"EC_results.npy"), EC_results)
np.save(os.path.join(home_dir,"EO_results.npy"), EO_results)

In [ ]:
#For getting at each individual measure, its some straight forward wrangling
np.array(EC_results['exponent']).squeeze(axis=1) #Replace the varialbe name and key name and you'll get out that data as participants X electrode.

In [ ]:
#Make some more human and machine readable headers
chans = (['Fp1',	'Fp2',	'F7',	'F3',	'Fz',	'F4',	'F8',	'FC5',	'FC1',	'FC2',	'FC6',	'T7',	'C3',	'Cz',	'C4',	'T8',	'CP5',	'CP1',	'CP2',	'CP6',	'AFz',	'P7',	'P3',	'Pz',	'P4',	'P8',	'PO9',	'O1',	'Oz',	'O2',	'PO10',	'AF7',	'AF3',	'AF4',	'AF8',	'F5',	'F1',	'F2',	'F6',	'FT7',	'FC3',	'FC4',	'FT8',	'C5',	'C1',	'C2',	'C6',	'TP7',	'CP3',	'CPz',	'CP4',	'TP8',	'P5',	'P1',	'P2',	'P6',	'PO7',	'PO3',	'POz',	'PO4',	'PO8',
])

#Make some more human and machine readable headers
exponent_header = [s + "_Exponent" for s in chans]
offset_header = [s + "_Offset" for s in chans]
fit_header = [s + "_Fit" for s in chans]

#Combine the headers
header_master = flatten([exponent_header,offset_header,fit_header])
header_eo = [s + "_EO" for s in header_master]
header_ec = [s + "_EC" for s in header_master]
header_eo = np.array(flatten([["ID"],header_eo]))[np.newaxis]
header_ec = np.array(flatten([["ID"],header_ec]))[np.newaxis]

#Get the data from the EO and EC dicts
ec_output = np.squeeze(np.concatenate([np.array(EC_results['exponent']).squeeze(axis=1),np.array(EC_results['offset']).squeeze(axis=1),np.array(EC_results['r2']).squeeze(axis=1)],axis=1))
ec_output = np.column_stack((np.array(EC_results['ID']),ec_output))
ec_output = np.concatenate((header_ec,ec_output),axis=0)

eo_output = np.squeeze(np.concatenate([np.array(EO_results['exponent']).squeeze(axis=1),np.array(EO_results['offset']).squeeze(axis=1),np.array(EO_results['r2']).squeeze(axis=1)],axis=1))
eo_output = np.column_stack((np.array(EO_results['ID']),eo_output))
eo_output = np.concatenate((header_eo,eo_output),axis=0)

#Save the data as a csv and numpy array
np.save(os.path.join(home_dir,"EC_Flat.npy"), ec_output)
np.save(os.path.join(home_dir,"EO_Flat.npy"), eo_output)

np.savetxt(os.path.join(home_dir,"EC_Flat.csv"), ec_output, delimiter=",",fmt = '%s')
np.savetxt(os.path.join(home_dir,"EO_Flat.csv"), eo_output, delimiter=",",fmt = '%s')

In [6]:
# Load the data from the saved .npy files 
# Pickle must be true. Then use [()] to get at the actual dictionary

EC_results = np.load(os.path.join(home_dir,"EC_results.npy"),allow_pickle=True).tolist()
EO_results = np.load(os.path.join(home_dir,"EO_results.npy"),allow_pickle=True).tolist()